In [1]:
const name = "LinearImplicit"
const residual_alphas = []
const implicit = true;

In [2]:
import SparseArrays: sparse
import NBInclude:@nbinclude
@nbinclude("Alpha.ipynb");

## Training

In [3]:
const alphas = [
    ["UniformBaseline"]
    ["PopularityBaseline"]
];

In [4]:
splits = []
@showprogress for split in all_splits
    push!(splits, read_alpha(alphas, split, implicit))
end
splits = reduce(cat, splits);

Progress: 100%|███████████████████████████| Time: 0:00:01 (94.16 ns/it)
Progress: 100%|█████████████████████████████████████████| Time: 0:06:47


In [5]:
preds = sparse(splits.user, splits.item, splits.rating);

In [6]:
function model(users, items)
    p = Array{eltype(preds)}(undef, length(users))
    Threads.@threads for i = 1:length(p)
        p[i] = preds[users[i], items[i]]
    end
    p
end
write_alpha(model, residual_alphas, implicit);

[ Info: 20220604 22:12:09 validation loss: 7.4751573, β: Float32[7.82374f-7, 0.99999917]
Progress: 100%|███████████████████████████| Time: 0:00:05 (30.05 ns/it)
[ Info: 20220604 22:14:19 training loss: 7.438401, β: Float32[7.82374f-7, 0.99999917]


In [7]:
_, β = regress(alphas, implicit)
write_params(Dict("β" => β, "alphas" => alphas));

In [8]:
# SOTA: 6.4687146803286275